In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import json
import gensim.downloader as api
from sklearn.metrics import f1_score
import numpy as np
from gensim.models import KeyedVectors
import pickle
import matplotlib.pyplot as plt

In [8]:
with open('Json Task1/NER_train.json', 'r') as f:
    task1_train_data = json.load(f)

with open('Json Task1/NER_val.json', 'r') as f:
    task1_val_data = json.load(f)

with open('Json Task1/NER_test.json', 'r') as f:
    task1_test_data = json.load(f)

with open('Json Task2/ATE_train.json', 'r') as f:
    task2_train_data = json.load(f)

with open('Json Task2/ATE_val.json', 'r') as f:
    task2_val_data = json.load(f)

with open('Json Task2/ATE_test.json', 'r') as f:
    task2_test_data = json.load(f)

with open('glove_embedding.pkl', 'rb') as pickle_file:
    glove_embeddings = pickle.load(pickle_file)
    
with open('fast_text_embedding.pkl', 'rb') as pickle_file:
    fast_text_embedding = pickle.load(pickle_file)

bio_mapping_task1 = {'B_ORG': 0, 'I_ORG': 1, 'B_RESPONDENT': 2, 'I_RESPONDENT': 3, 'B_JUDGE': 4, 'I_JUDGE': 5,
               'B_STATUTE': 6, 'I_STATUTE': 7, 'B_OTHER_PERSON': 8, 'I_OTHER_PERSON': 9, 'B_COURT': 10, 'I_COURT': 11,
               'B_GPE': 12, 'I_GPE': 13, 'B_PETITIONER': 14, 'I_PETITIONER': 15, 'B_WITNESS': 16, 'I_WITNESS': 17,
               'B_CASE_NUMBER': 18, 'I_CASE_NUMBER': 19, 'B_PRECEDENT': 20, 'I_PRECEDENT': 21, 'B_DATE': 22, 'I_DATE': 23,
               'B_PROVISION': 24, 'I_PROVISION': 25, 'O': 26}
bio_mapping_task2 = {'O' : 0, 'I' : 1, 'B' : 2}

In [9]:
word2vec_model = api.load('word2vec-google-news-300')

word2vec = torch.FloatTensor(word2vec_model.vectors)
word2vec_u_ = np.concatenate((word2vec, np.zeros((1, 300), dtype = 'float32')), axis = 0)

# Add an unknown token to the vocabulary
word_to_index = {word: index for index, word in enumerate(word2vec_model.index_to_key)}
word_to_index['<unk>'] = len(word_to_index)

# Example usage
unknown_token_index = word_to_index['<unk>']

In [10]:
class RNNModel(nn.Module):
    def __init__(self, embedding_dim, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(embedding_dim, 128, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, output_size)

    def forward(self, x):
        # x = self.embedding_layer(x)
        out, _ = self.rnn(x)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
    
class LSTMModel(nn.Module):
    def __init__(self, embedding_dim, output_size):
        super(LSTMModel, self).__init__()
        # self.embedding_layer = nn.Embedding.from_pretrained(pretrained_embedding, freeze=True)
        self.lstm = nn.LSTM(embedding_dim, 128, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, output_size)

    def forward(self, x):
        # x = self.embedding_layer(x)
        out, _ = self.lstm(x)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
    
class GRUModel(nn.Module):
    def __init__(self, embedding_dim, output_size):
        super(GRUModel, self).__init__()
        # self.embedding_layer = nn.Embedding.from_pretrained(pretrained_embedding, freeze=True)
        self.gru = nn.GRU(embedding_dim, 128, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, output_size)

    def forward(self, x):
        # x = self.embedding_layer(x)
        out, _ = self.gru(x)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

class Task_data(Dataset):
    def __init__(self, data, bio_index, embedding_type):
        self.data = data
        self.length = len(self.data)
        self.bio_index =  bio_index
        self.embedding_type = embedding_type

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        input_sequence = self.data[str(index)]['text'].split(' ')
        sentence_embeddings = []
        if self.embedding_type == "glove":
            sentence_embeddings = [glove_embeddings.get(word, np.zeros(300, dtype = 'float32')) for word in input_sequence]
        elif self.embedding_type == "word2vec":
            sentence_embeddings = [word2vec_u_[word_to_index.get(word, word_to_index['<unk>'])] for word in input_sequence]
        elif self.embedding_type == "fast_text":
            sentence_embeddings = [np.array(fast_text_embedding.get(word, np.zeros(300, dtype = 'float32')), dtype = 'float32') for word in input_sequence]

        sentence_embeddings = np.array(sentence_embeddings, dtype='float32')
        output_sequence = self.data[str(index)]['labels']
        output_labels = [self.bio_index[word] for word in output_sequence]

        # output_labels = np.array(output_labels, dtype='float32')
        return torch.tensor(sentence_embeddings), torch.tensor(output_labels)

In [11]:
def test_model(task, embedding_type, model, criterion, device, batch_size = 1):
    test_dataloader = None
    if task == 1:
        test_dataloader =  DataLoader(Task_data(task1_test_data, bio_mapping_task1, embedding_type), batch_size=batch_size, shuffle=False)

    elif task == 2:
        test_dataloader =  DataLoader(Task_data(task2_test_data, bio_mapping_task2, embedding_type), batch_size=batch_size, shuffle=False)

    total_test_loss = 0
    all_test_predictions = []
    all_test_targets = []

    with torch.no_grad():
        for test_inputs, test_targets in test_dataloader:
            test_inputs, test_targets = test_inputs.to(device), test_targets.to(device)
            test_outputs = model(test_inputs)

            loss = 0
            for i in range(test_outputs.size(1)):  # Iterate over time steps
                loss += criterion(test_outputs[:, i, :], test_targets[:, i])  

            total_test_loss += loss.item()

            all_test_predictions.extend(test_outputs.argmax(dim=2).view(-1).cpu().numpy())
            all_test_targets.extend(test_targets.view(-1).cpu().numpy())

        avg_test_loss = total_test_loss / len(test_dataloader)
        test_macro_f1 = f1_score(all_test_targets, all_test_predictions, average='macro')
    print(f'Test Loss: {avg_test_loss}, Test Macro F1-Score: {test_macro_f1}')

In [12]:
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [20]:
model = RNNModel(300,27)
task_ = 1
print("Task 1")
model.load_state_dict(torch.load('trained_models/t1_rnn_word2vec.pt'))
test_model(task = task_, embedding_type = "word2vec", model = model, criterion=criterion, device=device)
model.load_state_dict(torch.load('trained_models/t1_rnn_glove.pt'))
test_model(task = task_, embedding_type = "glove", model = model, criterion=criterion, device=device)
model.load_state_dict(torch.load('trained_models/t1_rnn_fast_text.pt'))
test_model(task = task_, embedding_type = "fast_text", model = model, criterion=criterion, device=device)

task_ = 2
print("Task 2")
model.load_state_dict(torch.load('trained_models/t2_rnn_word2vec.pt'))
test_model(task = task_, embedding_type = "word2vec", model = model, criterion=criterion, device=device)
model.load_state_dict(torch.load('trained_models/t2_rnn_glove.pt'))
test_model(task = task_, embedding_type = "glove", model = model, criterion=criterion, device=device)
model.load_state_dict(torch.load('trained_models/t2_rnn_fast_text.pt'))
test_model(task = task_, embedding_type = "fast_text", model = model, criterion=criterion, device=device)

Task 1
Test Loss: 10.049581709474023, Test Macro F1-Score: 0.5060528486601131
Test Loss: 9.693748308655838, Test Macro F1-Score: 0.5252059466991373
Test Loss: 8.536325708828194, Test Macro F1-Score: 0.5199378620639826
Task 2
Test Loss: 5.918120089692643, Test Macro F1-Score: 0.7261680735233901
Test Loss: 5.941616507057827, Test Macro F1-Score: 0.7664338339723803
Test Loss: 3.6531595318413497, Test Macro F1-Score: 0.7479234872811942


In [21]:
model = LSTMModel(300,27)
task_ = 1
print("Task 1")
task1_lstm_word2vec = torch.load('trained_models/t1_lstm_word2vec.pt')
model.load_state_dict(task1_lstm_word2vec)
test_model(task = task_, embedding_type = "word2vec", model = model, criterion=criterion, device=device)
task1_lstm_glove = torch.load('trained_models/t1_lstm_glove.pt')
model.load_state_dict(task1_lstm_glove)
test_model(task = task_, embedding_type = "glove", model = model, criterion=criterion, device=device)
task1_lstm_fast_text = torch.load('trained_models/t1_lstm_fast_text.pt')
model.load_state_dict(task1_lstm_fast_text)
test_model(task = task_, embedding_type = "fast_text", model = model, criterion=criterion, device=device)

task_ = 2
print("Task 2")
task2_lstm_word2vec = torch.load('trained_models/t2_lstm_word2vec.pt')
model.load_state_dict(task2_lstm_word2vec)
test_model(task = task_, embedding_type = "word2vec", model = model, criterion=criterion, device=device)
task2_lstm_glove = torch.load('trained_models/t2_lstm_glove.pt')
model.load_state_dict(task2_lstm_glove)
test_model(task = task_, embedding_type = "glove", model = model, criterion=criterion, device=device)
task2_lstm_fast_text = torch.load('trained_models/t2_lstm_fast_text.pt')
model.load_state_dict(task2_lstm_fast_text)
test_model(task = task_, embedding_type = "fast_text", model = model, criterion=criterion, device=device)

Task 1
Test Loss: 11.599112959540346, Test Macro F1-Score: 0.4699109343937621
Test Loss: 14.757322050976235, Test Macro F1-Score: 0.5115595178636165
Test Loss: 8.224305017250595, Test Macro F1-Score: 0.49904867618782833
Task 2
Test Loss: 3.4728042430346604, Test Macro F1-Score: 0.7574171957757222
Test Loss: 4.750821695468359, Test Macro F1-Score: 0.774386641843579
Test Loss: 3.245913795797472, Test Macro F1-Score: 0.7388548880395737


In [24]:
model = GRUModel(300,27)
task_ = 1
print("Task 1")
task1_gru_word2vec = torch.load('trained_models/t1_gru_word2vec.pt')
test_model(task = task_, embedding_type = "word2vec", model = task1_gru_word2vec, criterion=criterion, device=device)
task1_gru_glove = torch.load('trained_models/t1_gru_glove.pt')
test_model(task = task_, embedding_type = "glove", model = task1_gru_glove, criterion=criterion, device=device)
task1_gru_fast_text = torch.load('trained_models/t1_gru_fast_text.pt')
test_model(task = task_, embedding_type = "fast_text", model = task1_gru_fast_text, criterion=criterion, device=device)

task_ = 2
print("Task 2")
task2_gru_word2vec = torch.load('trained_models/t2_gru_word2vec.pt')
test_model(task = task_, embedding_type = "word2vec", model = task2_gru_word2vec, criterion=criterion, device=device)
task2_gru_glove = torch.load('trained_models/t2_gru_glove.pt')
test_model(task = task_, embedding_type = "glove", model = task2_gru_glove, criterion=criterion, device=device)
task2_gru_fast_text = torch.load('trained_models/t2_gru_fast_text.pt')
test_model(task = task_, embedding_type = "fast_text", model = task2_gru_fast_text, criterion=criterion, device=device)

Task 1
Test Loss: 9.373875467686338, Test Macro F1-Score: 0.5168654391871087
Test Loss: 8.077353415152622, Test Macro F1-Score: 0.5482033745408978
Test Loss: 8.06587407610965, Test Macro F1-Score: 0.5094257090085342
Task 2
Test Loss: 3.4817155589031556, Test Macro F1-Score: 0.7659194125872916
Test Loss: 3.547548048552438, Test Macro F1-Score: 0.8005179838913193
Test Loss: 3.169987880284103, Test Macro F1-Score: 0.7542284786600844
